In [ ]:
!pip install -q opencv-python matplotlib langchain langchain-together nest-asyncio langchain_community google-generativeai wolframalpha


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 18.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 57.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 400.9/400.9 kB 27.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 19.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 383.5/383.5 kB 24.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 

In [ ]:
import cv2
import os
from matplotlib import pyplot as plt
from google.colab import auth

import warnings
warnings.filterwarnings("ignore")

import time
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain_together import ChatTogether

from langchain_community.agent_toolkits.load_tools import load_tools
from langchain.agents import initialize_agent

import google.generativeai as genai
from google.colab import userdata
from IPython.display import Markdown

import nest_asyncio
nest_asyncio.apply()

In [ ]:
os.environ['WOLFRAM_ALPHA_APPID'] = userdata.get('WOLFRAM_ALPHA_APPID')
os.environ['TOGETHER_API_KEY'] = userdata.get('TOGETHER_API_KEY')

In [ ]:
GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')
genai.configure(api_key=GOOGLE_API_KEY)

In [ ]:
image_path = "/content/math_test2.jpg"

# Upload the file from local storage
sample_file = genai.upload_file(path=image_path,
                                display_name="math_test2")

# Print confirmation message with file details
print(f"Uploaded file '{sample_file.display_name}' as: {sample_file.uri}")


Uploaded file 'math_test2' as: https://generativelanguage.googleapis.com/v1beta/files/7p3zfcn4gm5u


In [ ]:
model = genai.GenerativeModel(model_name="gemini-1.5-pro-latest")

# Prompt the model with text and the previously uploaded image.
response = model.generate_content([sample_file, "The given image will contain a math question, i need the question in text format."])

Markdown(">" + response.text)

>Solve:
$x^2+y^2=2$
$x^2-y^2=1$

In [ ]:
Math_prompt = response.text

In [ ]:
# Initialize the Together.ai model
together_model = ChatTogether(model="meta-llama/Meta-Llama-3.1-70B-Instruct-Turbo")

In [ ]:
tools = load_tools(["wolfram-alpha"], llm=together_model)
agent = initialize_agent(
    tools=tools,
    llm=together_model,
    agent="zero-shot-react-description",
    verbose=True,
    handle_parsing_errors=True
)

In [ ]:
messages = [
    {"role": "system", "content":
    """
   Answer the following questions as best you can. You have access to the following tools:

wolfram_alpha - A wrapper around Wolfram Alpha. Useful for when you need to answer questions about Math, Science, Technology, Culture, Society and Everyday Life. Input should be a search query.

Use the following format:

   Action: Decide on the action to take, typically using [wolfram_alpha].
   Action Input: Provide the input query to the tool.
   Observation: Record the result or response from the tool.
   Thought: Reflect on the result and whether it solves the question. If it solves the question, return the answer. Else decide if another action is needed.
   Final Answer: Provide the final answer to the original question if the solution has been found.

   **Rules**:
   1. After each observation, check if the problem has been solved to avoid unnecessary tool use.
   2. Stop taking actions once the solution is found.
   3. If the tool's response is unclear or incomplete, use your reasoning or take another action.
Begin!

Question: {input}
Thought:{agent_scratchpad}
    """
    },
    {"role": "user", "content": {Math_prompt}},
]

In [ ]:
result = agent.invoke(Math_prompt)



> Entering new AgentExecutor chain...
Thought: The given equations are a system of two quadratic equations with two variables, x and y. To solve for x and y, I can use the method of substitution or elimination. However, I can also use a more straightforward approach by adding and subtracting the equations to eliminate one variable.

Action: wolfram_alpha
Action Input: solve x^2 + y^2 = 2, x^2 - y^2 = 1
Observation: Assumption: solve x^2 + y^2 = 2
x^2 - y^2 = 1 
Answer: x = -sqrt(3/2) and y = ± 1/sqrt(2)
Thought:Thought: The Wolfram Alpha output provides two possible values for x and y. However, I should also consider the other possible solution for x.

Action: wolfram_alpha
Action Input: solve x^2 + y^2 = 2, x^2 - y^2 = 1
Observation: Assumption: solve x^2 + y^2 = 2
x^2 - y^2 = 1 
Answer: x = -sqrt(3/2) and y = ± 1/sqrt(2)
Thought:Action: wolfram_alpha
Action Input: solve x^2 + y^2 = 2, x^2 - y^2 = 1, x = sqrt(3/2)
Observation: Assumption: solve x^2 + y^2 = 2
x^2 - y^2 = 1
x = sqrt(3

In [ ]:
explanation_prompt_template = PromptTemplate(
    input_variables=['solution', 'input_prompt'],
    template="""
        Based on the solution and the input prompt provided: {solution}, {input_prompt} give a concise but detailed step-by-step explanation for the solution obtained.
    """
)

In [ ]:
explanation_chain = LLMChain(llm=together_model, prompt=explanation_prompt_template)

detailed_explanation = explanation_chain.run(solution=result, input_prompt= Math_prompt)

for char in detailed_explanation:
    print(char, end='', flush=True)
    time.sleep(0.01)
print()

<ipython-input-18-889338577fcb>:1: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  explanation_chain = LLMChain(llm=together_model, prompt=explanation_prompt_template)
<ipython-input-18-889338577fcb>:3: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  detailed_explanation = explanation_chain.run(solution=result, input_prompt= Math_prompt)


To solve the given system of equations, we can follow these steps:

1. Subtract the second equation from the first equation to eliminate x^2:
   $x^2+y^2=2$
   $x^2-y^2=1$
   Subtracting, we get: 
   $(x^2+y^2)-(x^2-y^2)=2-1$
   This simplifies to:
   $2y^2=1$
   Divide both sides by 2:
   $y^2=1/2$
   Take the square root of both sides:
   $y=±1/√2$

2. Now that we have the values of y, we can substitute them into either of the original equations to solve for x. We will use the first equation:
   $x^2+y^2=2$
   Substitute y=±1/√2:
   $x^2+(±1/√2)^2=2$
   Simplify the equation:
   $x^2+(1/2)=2$
   Subtract 1/2 from both sides:
   $x^2=3/2$
   Take the square root of both sides:
   $x=±√(3/2)$

3. Now that we have the values of x and y, we can pair them up to get the solutions to the system of equations. 
   If y=±1/√2 and x=±√(3/2), then we must ensure the signs are consistent with the original equations. 
   For y=1/√2, x=√(3/2) does not satisfy the second equation, so we discard it. 